See the README.md for project description

From the 5-jupyter things article

First maintain your imports in one cell so you can quickly see dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Set up inline graphics

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

The data import is easy with pandas cause this is how it reads a csv.

We can cut the header of our cnee file with a simple `grep Cypress goodmouse-2017-10-28-1.dat > cnee.csv` then read that file

In [ ]:
trackpad = pd.read_csv("cnee.csv", header=None)

You can always bring up help for a methon with the `help()` function, like `help(pd.read_csv)`

Take a look at the data just to make sure its what we think it is

In [ ]:
trackpad

A the plot method is part of all data object types and is a quick way to look at the data.  A simple data point plot can be drawn with the 'o' style.

In [ ]:
trackpad[2].plot(style='o')

What we really need is to see this data in 2D since we are mapping a 2D surface.  This means we want the X values on the x-axis and Y as the y-axis.  Just looking at the raw data let's us see where all the trackpad was touched.

It doesnt' give us any insight into when or what kind of trajectories we developed here but this is where we can start to use our clustering algos to see how closely together the interaction occurred

In [ ]:
plt.scatter(trackpad[2],trackpad[3])

Let's look at some longer running input and see if more of the traces can be seen.

In [ ]:
tp2=pd.read_csv("cnee2.csv", header=None)

plt.scatter(tp2[2],tp2[3])

Our goal is to understand this data and get a sense both of what it means and if there's any pattern we can detect with inputs to distinguish as real and which are stray input from my hands touching the trackpad when i type.

The goal is to use [Scikit-learn](http://scikit-learn.org) for data discovery.  There is a very helpful [cheat sheet that maps out options](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) for data discovery based on the need to classify or predict values for a data set.

What's not clear is how to read this data.  It's time stamped so even the density is above is miss leading.

One way to think about this is that it is a collection of traces on the trackpad.  It would be good to know how many traces exists.  That is, to seperate the input into distinct time events that identify individual traces.   We could then compare the shapes of the traces and see if there's anything particular about intential action and the stray actions that muck with the trackpad.

A few things we can guess intuitively, if we just masked off a triangular area in the uppper left and right of the trackpad and ignored all input there, we might go a long way to reducing the errors with little effort.  Just place some non-conductive tape over the corners. Presto!

The down sides are, this solves the problem the easy way and we want to solve it the clever way.  Your hands can feel the tape boundary. The tape edges will likey gather lint and will eventually curl. You'll have to repeat the fix at some regular interval.  It lacks style.  It detracts from the appearance of the machine.

Letood 's set out to find how we can separate our data.  Asking google about [analyzing path data sets](https://www.google.com/search?client=ubuntu&channel=fs&q=analyzing+path+data+sets&ie=utf-8&oe=utf-8) doesn't seem to go in the direction we are going. 

It seems reasonable that things that happen close together in time are related.  Things that trace a geometric shape or some line would be more likely intentional than some random walk or touch points.

How do we identify these data boundaries?  Also, why do the random touch points seem to happen randomly. Or rather sometimes this problem is really a non-issue.

Tuning our questions to google to get more specific, we want to [isolate events in time series data](https://www.google.com/search?client=ubuntu&channel=fs&q=isolating+events+in+time+series+data&ie=utf-8&oe=utf-8).  This brings up a paper on [real-time pattern isolation and recognition](http://infolab.usc.edu/DocsDemos/mmm03-final.pdf) describing sensing user input and islating events in a virtual environemnt.  This feels more like the direction to head in.

There's a discussion thread of someone asking basically the same question [detecting events in time series data](https://cs.stackexchange.com/questions/61102/detecting-events-in-time-series-data). However, this discussion is focused on continuous signal time series data.

In our data set we don't have a continuous signal, data just appears when it occurs. This means we can easily detect the existance of "signal". It's just there.  What we really want is to separate our signal in time.  So go from one long stream of data to looking at the shapes of many different entries.

Tuning our question to [event detection in time series data brings up this helpful thread](https://cs.stackexchange.com/questions/57830/machine-learning-identify-patterns-in-time-series-data) a link to a survey paper on techniques.  There is a recent [survey paper on technics of machine learning for time series data](https://www.researchgate.net/publication/260086856_A_Review_of_Unsupervised_Feature_Learning_and_Deep_Learning_for_Time-Series_Modeling).  It suggests time series data analysis is a hard problem.  Dang.

Let's see if can just find the distribution of distances between events.  Maybe this would be helpful.


Column 7 is the data that looks like time stamps.
Pandas has a [diff() method](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.diff.html) for looking at time differences between events.

In [ ]:
tp2[7].diff()[1:]

This looks hopeful. But no idea if these are seconds or some other value.


In [ ]:
tp2[7].diff()[1:].describe()

So 75% of the data is above 13 seconds  of separation and 25% is less than 12 seconds.  Maybe this is useful.

How can we break or input traces into separate events?  What's the count of these events?


In [ ]:
plt.hist(tp2[7].diff()[1:], bins=4, log=1)

In [ ]:
pd.to_datetime(tp2[7])

So my time series data is not really date values.

Looking at a date in normal unix time `date +%s` shows the values are 1509301203
The dates above are 243084487

In [ ]:
243084487 - 1509301203

So need to understand my data a bit better.

In [ ]:
p2=pd.read_csv("cnee3.csv", header=None)

plt.scatter(tp2[2],tp2[3])

Take a break if things aren't working as expected

It's always good to clear your mind if it feels like you are at a dead end.  I took a run for this one.   Thinking about the data in this last picture, I realized I am getting the x input data.  That means it's mapped to my screen.  It doesn't show what my trackpad is receiving.  It shows what my desktop is recieving.  This means it's capturing, in some ways, more meaningful data.  It's certainly going to see valid input patterns.  You can see them in the verticle lines.  This is where I'm scrolling.   Most of my windows are full screen so literal positions on the screen that recieved mouse input.

The problem is, I don't what the screen's view of the input.  I want the trackpad's perspective.  It's that trackpad postitions that are getting touched once in a while and causing my focus to change to where ever the cursor happens to be.   I want the position on the trackpad location.  This should cause the stray inputs to have a focus around the upper corners.  I'm also like to find most touching in the center.  

Read the man page on the xev tool.  Pay attention when you read these. It's easy to overlook important details when your scanning.   It's for creating a window the prints xevents or debugging the events sent to a window this starts a window and prints events on the stdout of the caller

The stdout looks like this and makes it clear the time field is some sort of time indicator
```
PropertyNotify event, serial 37, synthetic NO, window 0x6e00001,
    atom 0x15d (_NET_WM_STATE), time 265181821, state PropertyNewValue
```

Even thought i don't know what the time represents the difference values are significant as spacing between events.

A second look

Should be able to get a better picture of the input distribution if I eliminate the very large outlier.

The distribution indicates a big gap above about 300k with a vally before the really big delay. Let's cut those outliers out and see what we get.

Pandas makes it easy to [drop data from an array](https://chrisalbon.com/python/pandas_dropping_column_and_rows.html).  We can just say keep only the rows below the cutoff.  Zooming in on this through a few iterations shows the most useful values are below 100 and generally indicates that our associated events can somewhat be considered to fall below that value.

In [ ]:
tpdiff=tp2[7].diff()[1:] # get a dedicated diff array 
tpclean=tpdiff[tpdiff < 1000]

plt.hist(tpclean, bins=30, log=1)

It would be better to discover the clusters using some more advance tooling.  Like maybe an clustering algorithm.

What kind of graph does this imply.  The difference is a distance between nodes.  We know a distance below 13 is very likely within a cluster, or sequence of moves.

So all nodes that are below 13 could be considered secquence.  Would clustering help make a better determination.

Let's skip that question for now and see if we can break our data structure into individual traces.

In [ ]:
tp3=pd.read_csv("hello.csv", header=None)

plt.scatter(tp3[2],tp3[3])